dim_Customer

In [0]:
%run "/Workspace/Utils/Utils"

In [0]:

from pyspark.sql import DataFrame, Window
from pyspark.sql import functions as F
from pyspark.sql.types import (
    IntegerType, StringType, TimestampType, StructType, StructField
)

In [0]:
#Loading Table

df = spark.table("adlslmcompany_silver.managed_silver.saleslt_customer")

In [0]:
#Displaying DF

df.display()

**Once the data has been cleaned, it should now be adapted to its final requirements, specifically as a dimension Customer table.**

In [0]:
def gold_clean_dimsalescustomer(df): 

    #Drop columns that will not be necessary for this dimension table
    df = df.drop("rowguid", "ModifiedDate", "bronze_ingestion_timestamp", "silves_transformed_timestamp" )

    # Add processed timestamp column

    df = df.withColumn("processed_timestamp", F.current_timestamp())

    #Cast to ensure datatype
    df = df.select(
         F.col('CustomerID').cast(IntegerType()).alias('CustomerID'),
         F.col('Gender').cast(StringType()).alias('Gender'), 
         F.col('SalesPerson').cast(StringType()).alias('SalesPerson'),
         F.col('CompanyName').cast(StringType()).alias('CompanyName'),
         F.col('HasPhone').cast(StringType()).alias('HasPhone'),
         F.col('HasEmail').cast(StringType()).alias('HasEmail'),
         F.col('processed_timestamp').cast(TimestampType()).alias('processed_timestamp')
                 )
    return df

In [0]:
#Defining expected schema
expected_schema = StructType([
    StructField("CustomerID", IntegerType(), False),             
    StructField("Gender", StringType(), True),                  
    StructField("SalesPerson", StringType(), True),
    StructField("CompanyName", StringType(), True),
    StructField("HasPhone", StringType(), False),  
    StructField("HasEmail", StringType(), False),
    StructField("processed_timestamp", TimestampType(), False) 
                            ])

In [0]:
gold_df = gold_clean_dimsalescustomer(df)

In [0]:
#Comparing lenghts

compare_lengths(df, gold_df)

In [0]:
#Checking the schema 
_validate_schema(gold_df, expected_schema)

**IMPORTANT: Please note that this is a simulated project; the upsert operation will be executed within this notebook. In a production environment, a dedicated notebook containing only the function and validations would be developed. All function notebooks would be orchestrated by Azure Data Factory (ADF) pipelines or Azure Databricks (ADB) workflows. The method of upsert may vary based on the utilization of auto loader, streaming, or Change Data Feed (CDF).**


In [0]:
#Loading into the Gold Layer 
target_table= "dim_customer"   

schema = "star_schema"

catalog = "adlscompany_gold"

primary_keys = ["CustomerID"]


upsert_table(gold_df, target_table, primary_keys, schema, catalog )